### Self consistent field

Now we have everything we need to run the SCF cycle! We'll combine here everything we have built so far to recap.

We also need to define some parameters for our SCF loop: a threshold for convergence and the maximum number of iterations, so that our loop has a definite end point. We'll print out the energy and the matrix norm of the difference between the new and old density matrices.

In [27]:
from pyscf import gto, scf, mp
from ase.units import Bohr
import numpy as np

# Define the system in PySCF and get all the molecular integrals
m = gto.Mole()
m.build(atom=f"H 0 0 0; H {1.4*Bohr} 0 0", basis="3-21g", spin=0, charge=0)
Tmat = m.intor("int1e_kin") # Integral matrix for kinetic energy
Smat = m.intor("int1e_ovlp") # Integral matrix for overlap integrals
Vmat = m.intor("int1e_nuc") # Integral matrix for electron-nucleus integrals 
Vee = m.intor("int2e") # Integral matrix for electron-electron integrals
nelec = m.nelectron # Number of electrons

In [28]:
# Orthogonalization matrix
s, U = np.linalg.eigh(Smat)
X = U / s**0.5

# Core Hamiltonian initial guess
Hcore = Tmat + Vmat
Hcore_ = np.dot(X.T, Hcore.dot(X)) # Transform from AO to MO basis
mo_eigs, mo_vecs = np.linalg.eigh(Hcore_) # Diagonalize the Hcore in MO basis
# Construct the density matrix from the initial guess
Cvec = np.dot(X, mo_vecs) # Transform the MO coefficients from MO to AO basis
Dmat = np.zeros((Smat.shape)) # Initialize the density matrix
for a in range(nelec//2): # Compute the density matrix in the AO basis. Since we use restricted HF, all spins are paired and only nelec/2 orbitals are considered
    Dmat += np.einsum('i,j->ij', Cvec[:, a], Cvec[:, a].T)

What exactly is the electronic energy, $E_0$, of the Hartree-Fock system? One might think, that since we have $N$ electrons, the total energy would be the sum of the eigenenergies for each electron, $\sum\limits_a^N \epsilon_a$. However, this is not correct: if we just add all the eigenvalues together, we end up counting the electron-electron interactions between the electrons twice (see Szabo and Ostlund, *Modern quantum chemistry* eqns: 3.81-3.82):

$$\sum\limits_a^N \epsilon_a = \sum\limits_a^N \left<a\left|h\right|a\right> + \sum\limits_a^N \sum\limits_b^N \left[ \left<aa\left|\right.bb\right> - \left<ab\left|\right.ba\right>\right] $$
$$E_0 = \sum\limits_a^N \left<a\left|h\right|a\right> \sum\limits_a^N + \frac{1}{2} \sum\limits_b^N \left[ \left<aa\left|\right.bb\right> - \left<ab\left|\right.ba\right>\right] $$
Above, $h$ is the core Hamiltonian and $a$ and $b$ correspond to the occupied orbitals. 



Let's have a look at the molecular orbitals and their energies. We will be using the Aufbau-principle to populate the molecular orbitals: populate the orbitals from the lowest orbital upwards. Since we are using the Numpy 'eigh', the eigenvalues are already ordered from the lowest eigenvalue to the highest.

In [36]:
maxiter = 50
conv_tol = 1e-6
iterstep = 0
conv = 1e0

print(f"{'Iteration':>10} {'Total Energy':>15} {'Dmat Difference':>18}")
while iterstep < maxiter:
    # Update the two-electron part
    Jmat = np.einsum('kl, ijkl->ij', Dmat, Vee, optimize=True) # Compute the Coulomb integrals
    Kmat = np.einsum('kl, ikjl->ij', Dmat, Vee, optimize=True) # Compute the exchange integrals

    Fmat = Hcore + 2*Jmat - Kmat # Construct the full Fock matrix
    #e_elec = np.trace(np.dot(2*Hcore + 2*Jmat - Kmat, Dmat)) # Compute the electronic energy
    e_elec = np.sum(Dmat * (Hcore + Fmat)) # Compute the electronic energy
    Dmat_old = Dmat.copy() # Store old density matrix

    Fmat_ = np.dot(X.T, Fmat.dot(X)) # Transform the Fockian from the AO to the orthonormal MO basis
    mo_eigs, mo_vecs = np.linalg.eigh(Fmat_) # Solve the Fockian
    Cvec = np.dot(X, mo_vecs) # Transform the MO coefficients from MO basis to AO basis

    # Construct a new density matrix
    Dmat *= 0
    for a in range(nelec//2):
        Dmat += np.einsum('i,j->ij', Cvec[:, a], Cvec[:, a].T)
        
    conv = np.linalg.norm(Dmat-Dmat_old) # Evaluate difference in the density matrices
    iterstep += 1
    print(f"{iterstep:>10} {e_elec:>15.8f} {conv:>18.3e}") # Print out energy
    if conv < conv_tol: # Check convergence. If converged, end cycle
        break 

 Iteration    Total Energy    Dmat Difference
         1     -1.83721908          2.541e-13


Next, we'll construct the density matrix for our initial guess from the molecular orbitals.

In [5]:
Cvec = np.dot(X, mo_vecs) # Transform the MO coefficients from MO to AO basis
Dmat = np.zeros((Smat.shape)) # Initialize the density matrix
for i in range(nelec//2): # Compute the density matrix in the AO basis. Since we use restricted HF, all spins are paired and only nelec/2 orbitals are considered
    Dmat += np.einsum('i,j->ij', Cvec[:, i], Cvec[i, :])
print(Dmat)

[[ 0.15723247 -0.13639791 -0.29254299 -0.40557068]
 [ 0.08583208 -0.07445864 -0.15969713 -0.22139813]
 [ 0.15723247 -0.13639791 -0.29254299 -0.40557068]
 [ 0.08583208 -0.07445864 -0.15969713 -0.22139813]]


In [6]:
Dmat = np.zeros((Smat.shape)) # Initialize the density matrix
for i in range(nelec//2): # Compute the density matrix in the AO basis. Since we use restricted HF, all spins are paired and only nelec/2 orbitals are considered
    Dmat += np.einsum('i,j->ij', mo_vecs[:, i], mo_vecs[i, :])
print(Dmat)
#Dmat = np.einsum('ij,ij,ij->ij', X, X.T, Dmat)
Dmat = np.dot(X.T, np.dot(Dmat, X))
print(Dmat)
print(np.dot(X.T, X))

[[ 4.75784716e-32 -1.14270858e-16  2.89880752e-32  1.85797316e-16]
 [ 3.55930763e-17 -8.54851201e-02  2.16857485e-17  1.38993495e-01]
 [ 6.65960034e-32 -1.59945921e-16  4.05748627e-32  2.60062131e-16]
 [-2.15201312e-16  5.16856420e-01 -1.31115431e-16 -8.40376430e-01]]
[[-6.56006633 -0.50289987 -0.90590507 -0.25163934]
 [-1.51092592 -0.11582878 -0.20864964 -0.05795801]
 [-0.90590507 -0.0694474  -0.12509995 -0.03474986]
 [-0.75603202 -0.05795801 -0.1044034  -0.02900083]]
[[ 9.24990970e+00  1.11022302e-15 -7.77156117e-16 -1.11022302e-16]
 [ 1.11022302e-15  2.11432470e+00  3.88578059e-16  5.55111512e-17]
 [-7.77156117e-16  3.88578059e-16  1.53210291e+00 -2.77555756e-17]
 [-1.11022302e-16  5.55111512e-17 -2.77555756e-17  3.61502982e-01]]


In [7]:
hf = m.RHF()
hf.verbose = 0
hf.max_cycle = -1
hf.init_guess = 'hcore'
hf.run()
print(Dmat)
print(hf.make_rdm1()/nelec)

[[-6.56006633 -0.50289987 -0.90590507 -0.25163934]
 [-1.51092592 -0.11582878 -0.20864964 -0.05795801]
 [-0.90590507 -0.0694474  -0.12509995 -0.03474986]
 [-0.75603202 -0.05795801 -0.1044034  -0.02900083]]
[[0.09358153 0.09121793 0.09358153 0.09121793]
 [0.09121793 0.08891402 0.09121793 0.08891402]
 [0.09358153 0.09121793 0.09358153 0.09121793]
 [0.09121793 0.08891402 0.09121793 0.08891402]]


In [12]:
maxiter = 50
conv_tol = 1e-6
iterstep = 0
conv = 1e0


Hcore = Tmat + Vmat
# Transform to orthonormal MO basis
Hcore_ = np.dot(X.T, Hcore.dot(X))
mo_eigs, mo_vecs = np.linalg.eigh(Hcore_)
Dmat = np.zeros((Smat.shape)) # Initialize the density matrix
Cvec = np.dot(X, mo_vecs)
for i in range(nelec//2): # Compute the density matrix. Since we use restricted HF, all spins are pair and only nelec/2 orbitals are considered
    Dmat += np.einsum('i,j->ij', Cvec[:, i], Cvec[:, i].T)
print(Dmat)
    
print(f"{'Iteration':>10} {'Total Energy':>15} {'Dmat Difference':>18}")
while iterstep < maxiter:
    Jmat = np.einsum('kl, ijkl->ij', Dmat, Vee, optimize=True) # Compute the Coulomb integrals
    #Kmat = np.einsum('kl, ilkj->ij', Dmat, Vee, optimize=True) # Compute the exchange integrals
    Kmat = np.einsum('kl, ikjl->ij', Dmat, Vee, optimize=True) # Compute the exchange integrals
        
    Fmat = Hcore + 2*Jmat - Kmat # Construct the full Fock matrix
    e_elec = np.trace(np.dot(2*Hcore + 2*Jmat - Kmat, Dmat))
    # Transform to orthonormal MO basis
    Fmat_ = np.dot(X.T, Fmat.dot(X))
    Dmat_old = Dmat.copy() # Store old density matrix
        
    mo_eigs, mo_vecs = np.linalg.eigh(Fmat_)
    Cvec = np.dot(X, mo_vecs)

    Dmat *= 0
    for i in range(nelec//2): # RHF
        Dmat += np.einsum('i,j->ij', Cvec[:, i], Cvec[:, i].T)
    conv = np.linalg.norm(Dmat-Dmat_old)
    iterstep += 1
    print(f"{iterstep:>10} {e_elec:>15.8f} {conv:>18.3e}")
    if conv < conv_tol:
        break

[[0.15723247 0.08583208 0.15723247 0.08583208]
 [0.08583208 0.04685512 0.08583208 0.04685512]
 [0.15723247 0.08583208 0.15723247 0.08583208]
 [0.08583208 0.04685512 0.08583208 0.04685512]]
 Iteration    Total Energy    Dmat Difference
         1     -1.78732579          1.533e-01
         2     -1.83611232          2.198e-02
         3     -1.83719589          3.175e-03
         4     -1.83721860          4.576e-04
         5     -1.83721907          6.595e-05
         6     -1.83721908          9.502e-06
         7     -1.83721908          1.369e-06
         8     -1.83721908          1.973e-07
